Purpose: Get to run basenji on GTEX-variant-data


original ressources: 
https://github.com/calico/basenji/blob/master/tutorials/
Preproc for CAGE-data
https://github.com/calico/basenji/blob/master/tutorials/preprocess.ipynb
ExpressionPrediction for Genes
https://github.com/calico/basenji/blob/master/tutorials/genes.ipynb


Kipoi:
Output-format for each bin:
https://storage.googleapis.com/131k/sample_wigs.txt

I/O Kipoi:

Inputs
Single numpy array
Name: seq
    Shape: (131072, 4) 
    Doc: * one-hot encoded DNA sequence * 4096bp starting flank sequence * 122880bp core sequence (960 * 128), predicted by the model in 128bp bins * 4096bp end flank sequence

Targets
Single numpy array
Name: genomic_features
    Shape: (960, 4229) 
    Doc: * 960 bins corresponding to 128bp regions on input sequence * 4229 different output tracks ordered according to https://storage.googleapis.com/131k/sample_wigs.txt


In [1]:
import kipoi
model = kipoi.get_model('Basenji')

Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.meta
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.index
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.data-00000-of-00001
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/const_feed_dict_pkl/3a76c37eb9ad255680ba774b110de1be
INFO:tensorflow:Restoring parameters from /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model


In [2]:
#does not work with any other batchsize...
pred = model.pipeline.predict_example(batch_size=2)

100%|██████████| 5/5 [00:14<00:00,  2.92s/it]


In [5]:
#input is a whole chromosome?
!head example/fasta_file | cut -c 1-30

head: cannot open ‘example/fasta_file’ for reading: No such file or directory


In [19]:
! head example/intervals_file 
#why are these intervals 51 bp instead of 128 and why are they not connected?

chr22	136018	136069
chr22	136351	136402
chr22	137749	137800
chr22	134503	134554
chr22	139010	139061
chr22	139157	139208
chr22	134638	134689
chr22	138908	138959
chr22	139449	139500
chr22	139450	139501


In [20]:
#Standard prediction process in kipoi-basenji
# Download example dataloader kwargs
dl_kwargs = model.default_dataloader.download_example('example')
# Get the dataloader and instantiate it
dl = model.default_dataloader(**dl_kwargs)
# get a batch iterator
it = dl.batch_iter(batch_size=2) #does not work with anything else
# predict for a batch
batch = next(it)
preds=model.predict_on_batch(batch['inputs'])

In [29]:
import numpy as np
tissuesB=lines = [line.rstrip('\n') for line in open("REP/tissueMapping/basenjiOut.txt")]
[cage for cage in list(zip(tissuesB,pred[0][0])) if cage[0].startswith("CAGE")]

[('CAGE:Hepatic Stellate Cells (lipocyte), donor2', 0.01165227),
 ('CAGE:Hepatic Stellate Cells (lipocyte), donor1', 0.0140641695),
 ('CAGE:Hepatic Stellate Cells (lipocyte), donor3', 0.016166987),
 ('CAGE:NK T cell leukemia cell line:KHYG-1', 0.042560045),
 ('CAGE:glioma cell line:GI-1', 0.009068377),
 ('CAGE:CD4+CD25+CD45RA- memory regulatory T cells, donor1', 0.053771533),
 ('CAGE:CD4+CD25+CD45RA- memory regulatory T cells, donor2', 0.04696772),
 ('CAGE:CD4+CD25+CD45RA- memory regulatory T cells, donor3', 0.11496869),
 ('CAGE:Skin - palm, donor1', 0.13509515),
 ('CAGE:carcinoid cell line:NCI-H727', 0.043501653),
 ('CAGE:Myoblast, donor3', 0.012716986),
 ('CAGE:Myoblast, donor2', 0.025209645),
 ('CAGE:Myoblast, donor1', 0.019006146),
 ('CAGE:glioblastoma cell line:A172, tech_rep2', 0.027876312),
 ('CAGE:glioblastoma cell line:A172, tech_rep1', 0.03055044),
 ('CAGE:Trabecular Meshwork Cells, donor3', 0.010913216),
 ('CAGE:Trabecular Meshwork Cells, donor1', 0.01911625),
 ('CAGE:caudat

In [3]:
#load samples of our data (Gtex)
import os
import sys

import pandas as pd
import numpy as np

import rep.preprocessing as p
from rep.constants import ANNDATA_CST as a

In [5]:
file = os.path.join(os.readlink(os.path.join("..","..","data")),"processed","gtex","recount","recount_gtex.h5ad")

gtex = p.load(file,backed="r")




In [8]:
# example how to split anndata by individual

sample = ['GTEX-QMR6']

gtex[:,gtex.var['Individual'].isin(sample)]   



# example how to split anndata by tissue

tissues = ['Lung','Blood']

gtex[:,gtex.var['Tissue'].isin(tissues)]  



# filter by subset of genes

genes = ['ENSG00000000003.14']

gtex[genes]

View of AnnData object with n_obs × n_vars = 1 × 8911 backed at '/s/project/rep/processed/gtex/recount/recount_gtex.h5ad'
    obs: 'gene_id', 'bp_length', 'symbol'
    var: 'project', 'sample', 'experiment', 'mapped_read_count', 'avg_read_length', 'Sample_Name', 'smatsscr', 'Parent_Tissue', 'Tissue', 'Individual', 'Gender', 'Assay_Type_s', 'Instrument_s', 'Indiv_Seq_Assay'

In [12]:
gtex[:,gtex.var['Tissue'].isin(tissues)]  


KeyError: "None of [['Lung', 'Blood']] are in the [index]"

In [14]:
import anndata
??anndata.read_h5ad

Signature: anndata.read_h5ad(filename, backed:Union[bool, str]=False)
Source:   
def read_h5ad(filename, backed: Union[bool, str] = False):
    """Read ``.h5ad``-formatted hdf5 file.

    Parameters
    ----------
    filename
        File name of data file.
    backed : {``False``, ``True``, ``'r'``, ``'r+'``}
        Load :class:`~anndata.AnnData` in `backed` mode instead of fully
        loading it into memory (`memory` mode). `True` and 'r' are
        equivalent. If you want to modify backed attributes of the AnnData
        object, you need to choose 'r+'.
    """

    if backed:
        # open in backed-mode
        return AnnData(filename=filename, filemode=backed)
    else:
        # load everything into memory
        d = _read_h5ad(filename=filename)
        return AnnData(d)
File:      /opt/modules/i12g/anaconda/3-5.0.1/envs/rep/lib/python3.6/site-packages/anndata/readwrite/read.py
Type:      function


In [50]:
#not needed anymore
!wget https://github.com/calico/basenji/raw/master/testdata/cage/h5/one_record_for_train_and_test.h5

--2019-01-22 16:00:25--  https://github.com/calico/basenji/raw/master/testdata/cage/h5/one_record_for_train_and_test.h5
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/calico/basenji/master/testdata/cage/h5/one_record_for_train_and_test.h5 [following]
--2019-01-22 16:00:25--  https://raw.githubusercontent.com/calico/basenji/master/testdata/cage/h5/one_record_for_train_and_test.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44759144 (43M) [application/octet-stream]
Saving to: ‘one_record_for_train_and_test.h5’

100%[======================================>] 44,759,144  99.2MB/s   in 0.4s   

2019-01-22 16:00:27 

In [54]:
import h5py
filename = 'one_record_for_train_and_test.h5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % list(f.keys()))
a_group_key = list(f.keys())[0]


Keys: ['pool_width', 'train_in', 'train_na', 'train_out', 'valid_in', 'valid_na', 'valid_out']


In [57]:

# Get the data
f['train_in']

<HDF5 dataset "train_in": shape (1, 262144, 4), type "|b1">